In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sqlalchemy import create_engine, Column, String, Integer, DateTime
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [3]:
DB_URL = (
    "postgresql://test:must_be_eight_characters"
    + "@example-1.cluster-cculi2axzscc.us-east-1.rds.amazonaws.com:5432/test"
)

engine = create_engine(DB_URL)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
# Create a base class for declarative models
Base = declarative_base()


# Define the User model
class User(Base):
    __tablename__ = "users"

    id = Column(String, primary_key=True)
    name = Column(String)
    password = Column(String)
    gender = Column(String)
    date_of_birth = Column(DateTime)


# Define the TaxDetails model
class TaxDetails(Base):
    __tablename__ = "tax_details"

    tax_id = Column(String, primary_key=True)
    user_id = Column(String)
    year = Column(Integer)
    income = Column(Integer)
    taxable_income = Column(Integer)
    location = Column(String)
    tax_amount = Column(Integer)

In [5]:
# fetch all users and save it as a dataframe
users = session.query(User).all()
users_df = pd.DataFrame([user.__dict__ for user in users])

In [7]:
# fetch all tax details and save it as a dataframe
tax_details = session.query(TaxDetails).all()
tax_details_df = pd.DataFrame([tax_detail.__dict__ for tax_detail in tax_details])

In [6]:
users_df.head()

,_sa_instance_state,date_of_birth,name,password,gender,id
0,<sqlalchemy.orm.state.InstanceState object at ...,1990-01-01,John Doe,password123,Male,1
1,<sqlalchemy.orm.state.InstanceState object at ...,1998-01-01,Dhrubo Kamal,$2b$12$H13QTJdzXESgvWcwQ3tS.uoXe7e0p6TGUEV/YTq...,male,0001
2,<sqlalchemy.orm.state.InstanceState object at ...,1998-01-01,Dhrubo Kamal,$2b$12$v7dQTrDddTmdXe7Sz5Z/H.Doo.iBmaifi.UDA64...,male,00011
3,<sqlalchemy.orm.state.InstanceState object at ...,1985-01-01,User Name,Password,male,6274836f-d459-4536-a85d-b6692bd341f0
4,<sqlalchemy.orm.state.InstanceState object at ...,1982-01-01,User Name,Password,male,52ee6704-75cb-4e3a-8c9d-7e178e72c1bf


In [8]:
tax_details_df.head()

,_sa_instance_state,income,tax_id,location,user_id,year,taxable_income,tax_amount
0,<sqlalchemy.orm.state.InstanceState object at ...,50000,1,New York,1,2023,40000,10000
1,<sqlalchemy.orm.state.InstanceState object at ...,901912,a50a6cf7-efd5-4f0f-b177-c9a8b99ce107,non_city,52ee6704-75cb-4e3a-8c9d-7e178e72c1bf,2015,551912,57786
2,<sqlalchemy.orm.state.InstanceState object at ...,864972,a1c492a6-5df9-42ea-a2f8-010f3c6add98,non_city,52ee6704-75cb-4e3a-8c9d-7e178e72c1bf,2016,514972,52245
3,<sqlalchemy.orm.state.InstanceState object at ...,902304,04b1c0ba-6966-46c8-8c3b-e57d99a124df,non_city,52ee6704-75cb-4e3a-8c9d-7e178e72c1bf,2017,552304,57845
4,<sqlalchemy.orm.state.InstanceState object at ...,861611,83ea4070-60b2-4c30-ab3d-6dfd931c41a5,non_city,52ee6704-75cb-4e3a-8c9d-7e178e72c1bf,2018,511611,51741


In [17]:
# perform join on both the dataframes, join key is user_id.id == tax_details.user_id
df = pd.merge(users_df, tax_details_df, left_on="id", right_on="user_id")

In [18]:
df

,_sa_instance_state_x,date_of_birth,name,password,gender,id,_sa_instance_state_y,income,tax_id,location,user_id,year,taxable_income,tax_amount
0,<sqlalchemy.orm.state.InstanceState object at ...,1990-01-01,John Doe,password123,Male,1,<sqlalchemy.orm.state.InstanceState object at ...,50000,1,New York,1,2023,40000,10000
1,<sqlalchemy.orm.state.InstanceState object at ...,1982-01-01,User Name,Password,male,52ee6704-75cb-4e3a-8c9d-7e178e72c1bf,<sqlalchemy.orm.state.InstanceState object at ...,901912,a50a6cf7-efd5-4f0f-b177-c9a8b99ce107,non_city,52ee6704-75cb-4e3a-8c9d-7e178e72c1bf,2015,551912,57786
2,<sqlalchemy.orm.state.InstanceState object at ...,1982-01-01,User Name,Password,male,52ee6704-75cb-4e3a-8c9d-7e178e72c1bf,<sqlalchemy.orm.state.InstanceState object at ...,864972,a1c492a6-5df9-42ea-a2f8-010f3c6add98,non_city,52ee6704-75cb-4e3a-8c9d-7e178e72c1bf,2016,514972,52245
3,<sqlalchemy.orm.state.InstanceState object at ...,1982-01-01,User Name,Password,male,52ee6704-75cb-4e3a-8c9d-7e178e72c1bf,<sqlalchemy.orm.state.InstanceState object at ...,902304,04b1c0ba-6966-46c8-8c3b-e57d99a124df,non_city,52ee6704-75cb-4e3a-8c9d-7e178e72c1bf,2017,552304,57845
4,<sqlalchemy.orm.state.InstanceState object at ...,1982-01-01,User Name,Password,male,52ee6704-75cb-4e3a-8c9d-7e178e72c1bf,<sqlalchemy.orm.state.InstanceState object at ...,861611,83ea4070-60b2-4c30-ab3d-6dfd931c41a5,non_city,52ee6704-75cb-4e3a-8c9d-7e178e72c1bf,2018,511611,51741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4931,<sqlalchemy.orm.state.InstanceState object at ...,1995-01-01,User Name,Password,male,458647be-e3b3-4c91-a52c-d9ca4c892e34,<sqlalchemy.orm.state.InstanceState object at ...,972897,945ebac4-b761-41c5-9b98-6aa1b96fa623,city,458647be-e3b3-4c91-a52c-d9ca4c892e34,2017,622897,68434
4932,<sqlalchemy.orm.state.InstanceState object at ...,1995-01-01,User Name,Password,male,458647be-e3b3-4c91-a52c-d9ca4c892e34,<sqlalchemy.orm.state.InstanceState object at ...,976710,b76bc525-e806-474f-aa9d-305df097e948,city,458647be-e3b3-4c91-a52c-d9ca4c892e34,2018,626710,69006
4933,<sqlalchemy.orm.state.InstanceState object at ...,1995-01-01,User Name,Password,male,458647be-e3b3-4c91-a52c-d9ca4c892e34,<sqlalchemy.orm.state.InstanceState object at ...,1000705,185da682-eb8a-4e75-ab43-91a55da3f18e,city,458647be-e3b3-4c91-a52c-d9ca4c892e34,2019,650705,72605
4934,<sqlalchemy.orm.state.InstanceState object at ...,1995-01-01,User Name,Password,male,458647be-e3b3-4c91-a52c-d9ca4c892e34,<sqlalchemy.orm.state.InstanceState object at ...,971006,5424effa-64bb-4198-bdee-689922d46bbe,city,458647be-e3b3-4c91-a52c-d9ca4c892e34,2020,621006,68150


In [19]:
# columns to keep for analysis - gender, income, id, location, tax_amount, taxable_income, year of df

df.drop("_sa_instance_state_x", axis=1, inplace=True)
df.drop("date_of_birth", axis=1, inplace=True)
df.drop("name", axis=1, inplace=True)
df.drop("password", axis=1, inplace=True)
df.drop("id", axis=1, inplace=True)
df.drop("tax_id", axis=1, inplace=True)
df.drop("user_id", axis=1, inplace=True)
df.drop("_sa_instance_state_y", axis=1, inplace=True)


In [22]:
df.head()

,gender,income,location,year,taxable_income,tax_amount
0,Male,50000,New York,2023,40000,10000
1,male,901912,non_city,2015,551912,57786
2,male,864972,non_city,2016,514972,52245
3,male,902304,non_city,2017,552304,57845
4,male,861611,non_city,2018,511611,51741


In [23]:
# save the dataframe as a csv file
df.to_csv("data.csv", index=False)